In [1]:
# Librerias tipicas para calculos y visualizaciones
import numpy as np
import matplotlib.pyplot as plt

# Para resolucion numerica
from scipy.optimize import least_squares

# Para matematica simbolica
import sympy as sp
from sympy.utilities.lambdify import lambdify
from sympy import print_latex

plt.style.use('fivethirtyeight')
sp.interactive.init_printing(use_latex='mathjax')

/Users/aliglara/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Enunciado

La reacción de síntesis catalítica de metanol puede ser representada mediante la siguiente ecuación 

\begin{equation*}
\text{CO(g)} + 2\text{H}_2\text{(g}) = \text{CH}_3\text{OH(g)}
\end{equation*}

Si se cuenta con una alimentación equimolar de los reactantes e igual a 5 moles a 200C. ¿Cuál será las fracciones molares si el sistema opera adiabáticamente a 10 bar y el catalizador permite que el sistema alcance el equilibrio sin que exista alguna otra reacción no deseada?

## Datos del problema

In [2]:
# Propiedades fisicoquimicas de los reactivos
# Calor y energia libre de Gibbs estandar de reaccion
# Coeficientes A y B para el calculo de las capacidades 
# calorificas a presion constante de los compuestos.

#                DHf(298)   DGf(298)  A     B x 10^2
#                 J/mol     J/mol
data = np.array([[-110530, -137160, 30.87, -1.285],
                 [0, 0, 27.14, 0.927],
                 [-200940, -162240, 21.15, 7.092]])

# Propiedades de operacion del reactor
TempInicial = 200 + 273.15       # Temperatura, K
PresionOp = 10                   # Presion, bar

# Datos de estado estandar para los calculos de equilibrio
Testandar = 298.15 
R = 8.314

# Datos de la reaccion
# Coeficientes estequimetricos y moles alimentados de los reactivos
#                      CO  H2 CH3OH
CoefMatrix = np.array([-1, -2, 1])  # negativo = reactivo, positivo = producto
molesAlime = np.array([5, 5, 0])

## Solucion

### Balance de moles

$$ n_i = n_{i0} + \nu_i\:\xi $$

In [3]:
xi, t, Tout = sp.symbols("xi, t, Tout", real=True)

ni = molesAlime + CoefMatrix * xi
nTotal = np.sum(ni)
yi = ni / nTotal

In [4]:
print('    CO          H2        CH3OH')
display([yi[n] for n in np.arange(3)])

    CO          H2        CH3OH


⎡ 5 - ξ    5 - 2⋅ξ      ξ    ⎤
⎢────────, ────────, ────────⎥
⎣10 - 2⋅ξ  10 - 2⋅ξ  10 - 2⋅ξ⎦

El grado de avance de esta reaccion debe satisfacer la restriccion que los moles finales deben ser iguales o mayores que cero, lo cual ocurre cuando

In [6]:
sol = sp.solve([ni[n] >= 0 for n in np.arange(3)])
display(sol)

0 ≤ ξ ∧ ξ ≤ 5/2

## Constante de equilibrio químico

### En funcion de la actividad de cada componente

Conociendo las expresiones de las composiciones que se alcanzarian en el equilibrio, se puede escribir la constante de equilibrio, K

$$ K(y) = \prod {a_i^{\nu_i}} = \prod {\frac{y_i\:P^{\nu_i}}{P^\circ}} $$

In [9]:
Ky = np.prod((yi * PresionOp)**CoefMatrix) 
display(Ky)

                2     
    ξ⋅(10 - 2⋅ξ)      
──────────────────────
             2        
100⋅(5 - 2⋅ξ) ⋅(5 - ξ)

### En funcion de la Energia Libre de Gibbs

Para conocer el valor del grado de avance se requiere conocer el valor de la constante de equilibrio, como una funcion de la temperatura de reaccion

$$ K(T) = \exp{\frac{-\Delta G^\circ(T)}{R\,T}} $$

Para lo cual es necesario resolver empleando el método de van't Hoff:

$$
\begin{align*}
&\ln K(T) = \ln K(T^\circ) + \frac{1}{R}\int_{T^\circ}^{T} \frac{\Delta H^\circ(t)}{t^2} \, dt\\
&\Delta H^\circ(T) = \Delta H^\circ(T^\circ) + \int_{T^\circ}^{T} {\Delta C_p^\circ(t) \, dt} \\
&\Delta C_p^\circ(t) = \sum{\nu_i\,C_{p,i}^\circ(T)} \equiv \sum{\nu_i\,A_i} + \sum{\nu_i\,B_i}\,T = \Delta A + \Delta B\,T
\end{align*}
$$

De esta manera, los calculos necesarios serian:

In [13]:
DA = np.sum(data[:, 2] * CoefMatrix)
DB = np.sum(data[:, 3] * 1e-2 * CoefMatrix)

DHT0 = np.sum(data[:, 0] * CoefMatrix)
DGT0 = np.sum(data[:, 1] * CoefMatrix)

KT0 = np.exp(-DGT0/R/Testandar)

In [14]:
print('El calor de reacción a 298.15 K es igual a {:.1f} J/mol'.format(DHT0))
print('El DG de reaccion a 298.15 K es igual a {:.1f} J/mol'.format(DGT0))
print('La constante de equilibrio químico a 298.15 K es igual a {:.3e}'.format(KT0))

El calor de reacción a 298.15 K es igual a -90410.0 J/mol
El DG de reaccion a 298.15 K es igual a -25080.0 J/mol
La constante de equilibrio químico a 298.15 K es igual a 2.478e+04


La expresión se Van't Hoff es una función que depende la temperatura, por lo cual se necesita una función de T

In [15]:
# Se definen dos variables para la expresion de van't Hoff
# t: variable de integracion
# T: variable que indica como cambia la temperatura dentro del reactor
t, T = sp.symbols("t, T", real=True)

Expresiones para evaluar las energias a cualquier temperatura T, empleando el modulo _integrate_ de _Sympy_ 

In [18]:
DHt = DHT0 + sp.integrate(DA + DB * t, (t, 298.15, T))

lnKe = np.log(KT0) + 1/R * (sp.integrate(DHt / (T**2), (T, 298.15, T)))

Ke = sp.exp(lnKe)

In [19]:
print_latex(Ke)

\frac{8494102314.01275 e^{0.00392290113062305 T + \frac{8928.03211147913}{T}}}{T^{7.69785903295646}}
